<a href="https://colab.research.google.com/github/CapitanMurloc/srt/blob/develop/Text_To_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/
#!mkdir tts
%env HF_HOME=/content/drive/MyDrive/tts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
env: HF_HOME=/content/drive/MyDrive/tts


In [2]:
%cd /content/drive/MyDrive/tts

/content/drive/MyDrive/tts


In [3]:
!pip install -q fairseq
!pip install -q huggingface-hub
!pip install -q sentencepiece

In [28]:
import torch
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
from fairseq.utils import move_to_cuda
import IPython.display as ipd

models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)

model = models[0]
model = model.to(torch.device("cuda:0"))

#device = next(model.parameters()).device

TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator([model], cfg)

text = "En este vídeo, estudiaremos la arquitectura del codificador, un ejemplo de arquitectura popular de sólo codificador"
sample = TTSHubInterface.get_model_input(task, text)
sample = move_to_cuda(sample)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

wav = wav.to('cpu')

ipd.Audio(wav, rate=rate)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.speech_to_text.hub_interface import S2THubInterface
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torchaudio


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/xm_transformer_600m-en_es-multi_domain",
    arg_overrides={"config_yaml": "config.yaml"},
)
model = models[0]
generator = task.build_generator([model], cfg)


# requires 16000Hz mono channel audio
audio, _ = torchaudio.load("/content/audio.flac")

sample = S2THubInterface.get_model_input(task, audio)
text = S2THubInterface.get_prediction(task, model, generator, sample)
print(text)

# speech synthesis
tts_models, tts_cfg, tts_task = load_model_ensemble_and_task_from_hf_hub(
  "facebook/tts_transformer-es-css10",
  arg_overrides={"vocoder": "griffin_lim", "fp16": False},
)
tts_model = tts_models[0]
TTSHubInterface.update_cfg_with_data_cfg(tts_cfg, tts_task.data_cfg)
tts_generator = tts_task.build_generator([tts_model], tts_cfg)

tts_sample = TTSHubInterface.get_model_input(tts_task, text)
wav, sr = TTSHubInterface.get_prediction(
    tts_task, tts_model, tts_generator, tts_sample
)

ipd.Audio(wav, rate=rate)
